# Testing the edge configuration package

In this notebook, the main goal is to test the edge configuration package created in notebook [30-CreatePipelinePackage](30-CreatePipelinePackage.ipynb)
driving it with the training data set which was used in notebook [20-CreateModel](20-CreateModel.ipynb).

The `LocalPipelineRunner` object takes the edge configuration package and extracts its components.
Once the components are extracted, you can run them individually by calling `run_component` with component name and structured input data.
The method builds a `venv` Python virtual environment for running the component and installs the required dependencies listed in `requirements.txt` for the component.
Once the virtual python environment is ready, the method executes and feeds the component with your test data.
The result will be the list of outputs of your component. If your component does not always produce an output, as is the case with a preprocessor aggregating a windowful of data, the output list will be shorter than the input list.

## Define a dataset to test the package
The goal here is to create a list of input data which the `process_input(data: dict)` method will be triggered with.  
For this reason, we read the json files and create the list of payloads as we did in notebook [20-CreateModel](20-CreateModel.ipynb)

In [ ]:
import json
from pathlib import Path

json_files = Path('../data/processed/example').glob('*.json')
inputs = []
for json_file in json_files:
    with open(json_file) as f:
        json_string = json.load(f)
        inputs.append({'json_data': json.dumps(json_string)})

## Test the pipeline locally
To do so we instantiate a `LocalPipelineRunner` object with the path of our configuration package, and a directory where we want to check the results.
This directory will contain both the extracted component and the created python virtual environment.
If the directory is not defined, a temporary directory is created and deleted after testing.

In [ ]:
from simaticai.testing.pipeline_runner import LocalPipelineRunner

test_dir = Path('../test')
package_path = Path('../packages/Batch-State-Identifier-edge_1.zip')

output = []
with LocalPipelineRunner(package_path, test_dir) as pipelineRunner:
    output = pipelineRunner.run_pipeline(inputs)  

## Check the results
The method returns with the calculated results which can be compared to the expected result.

In [ ]:
output